In [2]:
import pandas as pd
import numpy as np
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
import matplotlib.pyplot as plt

# Force usage of Mac GPU (MPS)
device_name = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"🚀 Device selected: {device_name}")

🚀 Device selected: mps


In [ ]:
def load_and_prep_data(file_path):
    df = pd.read_csv(file_path)
    
    # 1. Fix TotalCharges
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce").fillna(0)
    
    # 2. Drop ID
    if 'customerID' in df.columns:
        df = df.drop(columns=['customerID'])
        
    # 3. Encode Target (Yes/No -> 1/0)
    target = 'Churn'
    if df[target].dtype == 'object':
        df[target] = df[target].map({'Yes': 1, 'No': 0})
    
    # 4. Define Lists for TabNet
    nunique = df.nunique()
    types = df.dtypes
    categorical_columns = []
    categorical_dims = {}
    
    # 5. Preprocessing Loop
    for col in df.columns:
        if col == target:
            continue
        
        # LOGIC: Treat as Categorical if Object OR <50 unique values (covers Binary & small ints)
        if types[col] == 'object' or nunique[col] < 50:
            df[col] = df[col].fillna("Unknown")
            l_enc = LabelEncoder()
            # Convert to string first to handle mixed types
            df[col] = l_enc.fit_transform(df[col].astype(str).values)
            categorical_columns.append(col)
            categorical_dims[col] = len(l_enc.classes_)
        else:
            # Treat as Numerical -> Fill with Mean
            df[col] = df[col].fillna(df[col].mean())
            
    return df, categorical_columns, categorical_dims

# Execute
FILE_PATH = "/Users/jonaslorler/master-thesis-uq-churn/data/raw/kaggle_churn/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df, cat_cols, cat_dims = load_and_prep_data(FILE_PATH)

print(f"✅ Data Processed. Shape: {df.shape}")
print(f"Categorical columns found: {len(cat_cols)}")

✅ Data Processed. Shape: (7043, 20)
Categorical columns found: 17


In [ ]:
target = 'Churn'
features = [col for col in df.columns if col != target]

# Get indices of categorical columns for TabNet
cat_idxs = [i for i, f in enumerate(features) if f in cat_cols]
cat_dims_list = [cat_dims[f] for f in features if f in cat_cols]

# Force features to be 32bit floats for MAC
X = df[features].values.astype('float32') 
y = df[target].values

# 1. Split Train (80%) vs Temp (20%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. Split Temp into Valid (10%) and Test (10%)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Train Shape: {X_train.shape}")
print(f"Valid Shape: {X_valid.shape}")
print(f"Test Shape:  {X_test.shape}")
print(f"Data Type:   {X_train.dtype}") # Should say float32

Train Shape: (5634, 19)
Valid Shape: (704, 19)
Test Shape:  (705, 19)
Data Type:   float32


In [8]:
# Initialize TabNet
clf = TabNetClassifier(
    cat_idxs=cat_idxs,
    cat_dims=cat_dims_list,
    cat_emb_dim=1,           # Low dimension is often sufficient for tabular
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax',      # "entmax" or "sparsemax"
    device_name=device_name, # <--- Uses your Mac GPU
    verbose=1
)

# Fit the model
print("🚀 Training started...")
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],     # Tracking AUC
    max_epochs=50,           # 50 is a good start for baseline
    patience=15,             # Stop if no improvement for 15 epochs
    batch_size=1024, 
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)
print("✅ Training complete.")

/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : mps
  warnings.warn(f"Device used : {self.device}")
/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


🚀 Training started...


/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


epoch 0  | loss: 0.64894 | train_auc: 0.51074 | valid_auc: 0.49209 |  0:00:00s


/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


...

/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


epoch 43 | loss: 0.35883 | train_auc: 0.87517 | valid_auc: 0.82724 |  0:00:18s


/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


epoch 44 | loss: 0.36853 | train_auc: 0.87906 | valid_auc: 0.82535 |  0:00:18s
epoch 45 | loss: 0.35971 | train_auc: 0.88129 | valid_auc: 0.82253 |  0:00:19s

Early stopping occurred at epoch 45 with best_epoch = 30 and best_valid_auc = 0.83806
✅ Training complete.


/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/jonaslorler/master-thesis-uq-churn/.venv/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [9]:
def calculate_top_decile_lift(y_true, y_pred_proba):
    # Create a temporary DataFrame
    df_res = pd.DataFrame({'y': y_true, 'p': y_pred_proba})
    
    # Sort by predicted probability (highest risk first)
    df_res = df_res.sort_values('p', ascending=False)
    
    # Select the top 10%
    top_decile = df_res.head(len(df_res) // 10)
    
    # Calculate churn rates
    actual_churn_rate = y_true.mean()
    top_decile_churn_rate = top_decile['y'].mean()
    
    # Calculate Lift
    return top_decile_churn_rate / actual_churn_rate

# Predict on Test Set
preds = clf.predict_proba(X_test)[:, 1]

# Calculate Metrics
auc_roc = roc_auc_score(y_test, preds)
auc_pr = average_precision_score(y_test, preds)
lift = calculate_top_decile_lift(y_test, preds)

print("\n--- 📊 Thesis Baseline Results ---")
print(f"AUC-ROC: {auc_roc:.4f}")
print(f"AUC-PR:  {auc_pr:.4f} (Primary Metric)")
print(f"Lift:    {lift:.4f}   (Primary Metric)")


--- 📊 Thesis Baseline Results ---
AUC-ROC: 0.7866
AUC-PR:  0.5357 (Primary Metric)
Lift:    2.3697   (Primary Metric)
